In [1]:
import qiskit
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import Parameter, ParameterVector
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
from qiskit import transpile, assemble
from qiskit.visualization import *
from qiskit.visualization import plot_bloch_multivector
from qiskit.aqua.operators import Z, Y, X
from qiskit.aqua.operators import StateFn
import numpy as np

C:\Users\julia\anaconda3\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Original forward function. We define the circuit, apply H gates to all qubits, apply Ry gates for the 'random quantum circuit', then measure. The output is the probability of each qubit being in the 1 state.

In [5]:
# Define rotation angles
params = np.array([0,np.pi/2,np.pi,np.pi/4])

# Define Hamiltonina
H = Z ^ Z ^ Z ^ Z

def forward(params):
    
    # Define 4 qubit quantum circuit
    qc = QuantumCircuit(4)
    
    # Apply H gates
    qc.h(range(4))
    
    # Apply random quantum circuit
    # Use ry rotations w/ rotation angles specified by components of the input vector
    for i in range(len(params)):
        qc.ry(params[i],i)
    
    qobj = assemble(qc, shots=8192)
    
    # Take measurement
    psi = StateFn(H, is_measurement=True) @ StateFn(qc)
    expectation = psi.eval()
    qc.measure_all()
    
    # Draw
    print(qc.draw())
    
    # Run circuit on Aer backend
    backend = Aer.get_backend('aer_simulator')
    qobj = transpile(qc, backend)
    job = backend.run(qobj)
    result = job.result()
    counts = result.get_counts()
    
    # Convert counts to probabilities in 2d array
    totalCount = 0
    exps = np.zeros(4)
    
    # Determine total count to convert to probabilities
    for state, prob in counts.items():
        totalCount += prob
      
    # Sum up probabilities for each qubit to get expectation value
    for state, prob in counts.items():
        prob /= totalCount
        
        # iterate over each qubit
        i = 0
        for c in state:
            qubit = float(c)
            exps[i] = exps[i] + qubit*prob
            i = i + 1

    #print(counts)
    print(exps)

In [6]:
forward(params)

        ┌───┐ ┌───────┐  ░ ┌─┐         
   q_0: ┤ H ├─┤ Ry(0) ├──░─┤M├─────────
        ├───┤┌┴───────┴┐ ░ └╥┘┌─┐      
   q_1: ┤ H ├┤ Ry(π/2) ├─░──╫─┤M├──────
        ├───┤└┬───────┬┘ ░  ║ └╥┘┌─┐   
   q_2: ┤ H ├─┤ Ry(π) ├──░──╫──╫─┤M├───
        ├───┤┌┴───────┴┐ ░  ║  ║ └╥┘┌─┐
   q_3: ┤ H ├┤ Ry(π/4) ├─░──╫──╫──╫─┤M├
        └───┘└─────────┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════╩══╩══╩══╩═
                            0  1  2  3 
[0.86816406 0.48925781 1.         0.50195312]


In [4]:
# Desired format
def get_quanv_fn(kernel_size=2, backend=None, shots=1024, threshold=127, ansatz=''):  # note: threshold and ansatz are just suggestions
    # Instantiate quantum circuit
    # create param vector 
    # create input param vector
    # apply appropriate gates
    
    
    def execute_qc(input_data, params):
        # bind data to circuit
        # execute
        # extract ouput expectations
        
        return results
    
    return execute_qc